In [19]:
import requests
import time
import numpy as np
import pandas as pd
import datetime as dt

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (18, 8)
cat_colors = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', 
              '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5', 
              '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', 
              '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']

In [2]:
robots = requests.get('http://www.rappler.com/robots.txt')
crawl_delay = [int(row.split(':')[1]) for row in robots.text.split('\n') if 'Crawl-delay' in row]
crawl_delay = crawl_delay[0] if crawl_delay else 30
crawl_delay = 0

In [3]:
r_phl1 = requests.get('http://ph.rappler.com/stubs/phl1.json')
time.sleep(crawl_delay)
r_phl2 = requests.get('http://ph.rappler.com/stubs/phl2.json')

In [4]:
phl1 = r_phl1.json()
phl2 = r_phl2.json()

In [5]:
provinces = [province['properties']['name'] 
             for province in phl1['objects']['subunits']['geometries'] 
             if 'properties' in province]

In [6]:
cities = [province['properties']['name'] 
          for province in phl2['objects']['subunits']['geometries'] 
          if 'properties' in province]

In [7]:
data = []
endpoint = 'http://ph.rappler.com/api/Locations/name/'
for province in provinces:
    print 'getting %s ...' % province
    r = requests.get(endpoint, params={'q': province.replace('-', ' ')})
    if r.status_code == 200:
        row = r.json()
        if row:
            data.append(row[0])
    time.sleep(crawl_delay)

getting Tawi Tawi ...
getting Bohol ...
getting Cebu ...
getting Negros Oriental ...
getting Siquijor ...
getting Negros Occidental ...
getting Basilan ...
getting Zamboanga del Norte ...
getting Zamboanga Sibugay ...
getting Zamboanga del Sur ...
getting Misamis Occidental ...
getting Sulu ...
getting Aklan ...
getting Antique ...
getting Capiz ...
getting Iloilo ...
getting Guimaras ...
getting Palawan ...
getting Romblon ...
getting Albay ...
getting Camarines-Norte ...
getting Camarines-Sur ...
getting Catanduanes ...
getting Masbate ...
getting Sorsogon ...
getting Abra ...
getting Batanes ...
getting Cagayan ...
getting Apayao ...
getting Ilocos-Norte ...
getting Ilocos-Sur ...
getting Aurora ...
getting Isabela ...
getting Ifugao ...
getting Mountain-Province ...
getting Nueva-Vizcaya ...
getting Quirino ...
getting Bataan ...
getting Tarlac ...
getting Nueva-Ecija ...
getting Pampanga ...
getting Benguet ...
getting Zambales ...
getting La-Union ...
getting Pangasinan ...
getti

In [ ]:
requests.a

In [8]:
for city in cities:
    print 'getting %s ...' % city
    r = requests.get(endpoint, params={'q': city})
    if r.status_code == 200:
        row = r.json()
        if row:
            data.append(row[0])
    time.sleep(crawl_delay)

getting Mandaluyong City ...
getting Manila ...
getting Navotas City ...
getting Caloocan City ...
getting Malabon City ...
getting Valenzuela City ...
getting Quezon City ...
getting Marikina-City ...
getting San Juan City ...
getting Pasig-City ...
getting Makati-City ...
getting Pasay-City ...
getting Parañaque-City ...
getting Las Piñas City ...
getting Muntinlupa City ...
getting Taguig City ...
getting Pateros ...
getting Lucena City ...


In [9]:
geo_table = pd.DataFrame([row.values() for row in data], columns=data[0].keys())

In [10]:
vote_count_endpoint = 'http://ph.rappler.com/api/Votes/live/transmission/location/id/{id}'

In [12]:
voter_data = []
for _id in geo_table['id'].sort_values():
    r = requests.get(vote_count_endpoint.format(id=_id))
    print 'getting %s ...' % _id
    try:
        geo_data = r.json()
        if geo_data:
            geo_data = geo_data[0]
            for geo_row in geo_data['transmission_data']: 
                for attribute in ['location_id', 'location_name', 'total_registered_voters', 'total_votes_count']:
                    geo_row[attribute] = geo_data[attribute]
                voter_data.append(geo_row)
    except:
        pass
    time.sleep(crawl_delay)

getting 21 ...
getting 22 ...
getting 23 ...
getting 24 ...
getting 25 ...
getting 26 ...
getting 27 ...
getting 28 ...
getting 29 ...
getting 30 ...
getting 31 ...
getting 32 ...
getting 33 ...
getting 34 ...
getting 35 ...
getting 36 ...
getting 37 ...
getting 38 ...
getting 39 ...
getting 40 ...
getting 41 ...
getting 42 ...
getting 43 ...
getting 44 ...
getting 45 ...
getting 46 ...
getting 47 ...
getting 48 ...
getting 49 ...
getting 50 ...
getting 51 ...
getting 52 ...
getting 53 ...
getting 54 ...
getting 55 ...
getting 56 ...
getting 57 ...
getting 58 ...
getting 59 ...
getting 60 ...
getting 61 ...
getting 62 ...
getting 63 ...
getting 63 ...
getting 64 ...
getting 65 ...
getting 66 ...
getting 67 ...
getting 67 ...
getting 68 ...
getting 69 ...
getting 70 ...
getting 71 ...
getting 72 ...
getting 73 ...
getting 74 ...
getting 75 ...
getting 76 ...
getting 77 ...
getting 78 ...
getting 79 ...
getting 80 ...
getting 81 ...
getting 82 ...
getting 83 ...
getting 84 ...
getting 85

In [13]:
votes_per_province = pd.DataFrame([row.values() for row in voter_data], columns=voter_data[0].keys())

In [14]:
votes_per_province['date'] = votes_per_province['date'].apply(lambda x: dt.datetime.fromtimestamp(x/1000))

In [15]:
plot_data = votes_per_province.pivot_table('percent', 'date', 'location_name', max)

In [16]:
geo_table.to_csv('rappler_geo.csv', index=False, encoding='utf-8')

In [17]:
votes_per_province.to_csv('rappler_tx_data.csv', index=False, encoding='utf-8')